# Массовое скачивание фотографий из групп ВКонтакте с использованием API

In [1]:
import os
from re import split
from time import sleep
from datetime import datetime
import vk
import requests

# secret.py - файл, содержащий переменную token, где записан ключ API
from secret import token

## Настройки программы
- public_link - полная ссылка на группу или паблик
- album_id - id альбома ('wall' - стена сообщества)
- rev - порядок загрузки (0 - хронологический, 1 - антихронологический)
- start_offset - с какой по счету фотографии начать загрузку
- phs_num - сколько фотографий нужно загрузить
- download_dir_name - путь к директории загрузки (если пустая строка, то файлы сохранятся в папке с кратким названием сообщества)

In [2]:
public_link = 'https://vk.com/ru2ch'
album_id = 'wall'
rev = 1
start_offset = 0
phs_num = 50
download_dir_name = ''

### Авторизируемся в ВК

In [3]:
session = vk.Session(access_token=token)
vk_api = vk.API(session)

### Получаем id сообщества

In [4]:
public_name = split(r'/', public_link)[-1]
if download_dir_name == '':
    download_dir_name = public_name
public_data = vk_api.groups.getById(group_id=public_name, v=5.102)
public_id = public_data[0]['id']
public_data

[{'id': 22751485,
  'name': 'Двач',
  'screen_name': 'ru2ch',
  'is_closed': 0,
  'type': 'page',
  'photo_50': 'https://sun4-12.userapi.com/c850536/v850536351/88c3/9pNLLDJ05ns.jpg?ava=1',
  'photo_100': 'https://sun4-16.userapi.com/c850536/v850536351/88c2/Fvb6jZI55bU.jpg?ava=1',
  'photo_200': 'https://sun4-11.userapi.com/c850536/v850536351/88c1/QJvU4toWrRg.jpg?ava=1'}]

### При помощи API получаем список URL изображений и записываем в файл
API не разрешает получать ссылки более чем на тысячу фотографий за один раз, поэтому программа сделает несколько запросов.

In [5]:
photo_links = []
offset = start_offset
now = datetime.now().strftime('%d-%m-%Y-%H-%M')

remain_num = phs_num
for i in range(int(phs_num / 1000) + 1):
    if remain_num >= 1000:
        download_num = 1000
    else:
        download_num = remain_num
    remain_num -= download_num
    wall_phs = vk_api.photos.get(owner_id=-public_id, album_id=album_id, v=5.102, count=download_num, offset=offset, rev=rev)
    for photo in wall_phs['items']:
        photo_links.append(photo['sizes'][-1]['url'])
    offset += download_num
    
with open(f'dump_{public_id}_{album_id}_{start_offset}_{phs_num}_{now}.txt', 'w') as file:
    file.writelines('%s\n' % link for link in photo_links)

### Загружаем фотографии
За счёт хранения ссылок в файле можно будет возобновлять загрузку после прерывания - ранее скачанные файлы в данную директорию загружаться не будут. Для перезапуска процесса скачивания после повторного открытия Jupyter-ноутбука нужно **указать вручную путь к файлу с URL**.

In [6]:
url_filename = f'dump_{public_id}_{album_id}_{start_offset}_{phs_num}_{now}.txt'
with open(url_filename, 'r') as file:
    photo_links = file.read().splitlines()

if not os.path.exists(download_dir_name):
    os.makedirs(download_dir_name)

ld = os.listdir(download_dir_name)
for link in photo_links:
    ce = True
    filename = split('/', link)[-1]
    if filename not in ld:
        while ce:
            try:
                img_data = requests.get(link).content
                ce = False
            except requests.exceptions.ConnectionError:
                sleep(5)
        with open(os.path.join(download_dir_name, filename), 'wb') as handler:
            handler.write(img_data)

### Проверим результат

In [7]:
print(f'В директории "{download_dir_name}" {len(os.listdir(download_dir_name))} файлов')

В директории "ru2ch" 50 файлов
